In [2]:
from utilities import *
import os

fout = 'output_porosity_wave1/'

if not os.path.exists(fout):
    os.makedirs(fout)

# model parameters
mu          = 0.1
lmbda       = mu*3

compr       = 100
mob         = 1e-6          # Mobility
Mdarcy      = 10.0
viscs       = 1.0           # Solid Stokes viscosity
viscf       = 10.0          # Fluid Stokes viscosity
phi         = 1.0           # Porosity
g0          = 0.5
tau0        = 2e-1*100
eps         = 1.0
eps1        = 7e-1

# parameters for initial data
alpha       = 0.25
z0          = 0.1e-2
mag         = 1e-2

pars = {}
pars['mu']     = mu 
pars['lmbda']  = lmbda 
pars['compr']  = compr
pars['mob']    = mob
pars['Mdarcy'] = Mdarcy
pars['viscs']  = viscs
pars['viscf']  = viscf
pars['phi']    = phi
pars['g0']     = g0
pars['tau0']   = tau0
pars['eps']    = eps
pars['eps1']   = eps1
pars['alpha']  = alpha
pars['z0']     = z0
pars['mag']    = mag

write_dictionary(fout + 'parameters.json',pars)

In [3]:
from fenics import *
from mshr import *
import numpy as np
from pylab import plt
import logging
from utilities import *

parameters["form_compiler"]["quadrature_degree"] = 3

def project_vectorfield_with_bc(u):
    Vtemp = VectorFunctionSpace(mesh,'CG',2) 
    
    unew,dunew = TrialFunction(Vtemp),TestFunction(Vtemp)
    
    Lhs = inner(unew,dunew)*dx
    rhs = inner(u,dunew)*dx
    
    unew = project(u,Vtemp)
    
    bc0t     = DirichletBC(Vtemp.sub(0),Constant(0), 'on_boundary && (near(x[0],  0,1e-3) || near(x[0],20,1e-3))')      # left & right
    bc1t     = DirichletBC(Vtemp.sub(1),Constant(0), 'on_boundary && (near(x[1],-40,1e-3) || near(x[1],40,1e-3))')      # top & bottom
    bcu      = [bc0t,bc1t]
    
    solve(Lhs==rhs,unew,bcu)
    
    return unew

def get_energy(us, uf, zeta,g0):
    # us, uf, zeta, muz, lmbda = split(q)
    d = 2
    I = Identity(d)
    Fs = inv(I - grad(us))
    Ff = inv(I - grad(uf))
    Js = det(Fs)
    Jf = det(Ff)
    Cs     = Fs.T*Fs
    p = zeta - (Js-1) # inner(abiot,Fs-I)
    
    eps2 = 1e-2
    
    Hs      = mu/2*(tr(Cs-I) - 2*ln(Js)) + lmbda * (Js - 1)**2
    Hmix    = compr*p**2/2
    Hgr     = zeta*inner(Constant((0,g0)),uf) 
    Hreg    = eps/2*inner(grad(zeta),grad(zeta)) + eps1*zeta*(ln(zeta)-1) # + eps2 * inner(Fs,Fs)
    Htot    = Hs + Hmix + Hgr + Hreg

    return Htot*dx

# bilinear forms for saddle point structure
def s(us,uf,zeta,dot_us,dot_uf,xius,xiuf):
    I = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    Fs = inv(I-grad(us))
    Js = det(Fs)
    
    # s_darcy   = (Mdarcy*sqrt( (z0/(zeta+z0))**2 + 1e-5 ))*phi*inner((Fs*dot_us-Ff*dot_uf),(Fs*xius - Ff*xiuf))
    s_darcy   = (Mdarcy*sqrt( (z0/(zeta+z0))**4 + 1e-5 ))*phi*inner((Fs*dot_us-Ff*dot_uf),(Fs*xius - Ff*xiuf))
    
    ss_stokes = viscs*inner(grad(Fs*dot_us),grad(Fs*xius))
    sf_stokes = viscf*inner(grad(Ff*dot_uf),grad(Ff*xiuf))
    blfs      = (s_darcy + ss_stokes + sf_stokes)
    return blfs

def a(muz,xiz):
    return mob*inner(grad(muz),grad(xiz))

def b(uf,zeta,dot_zeta,duf,dzeta):
    I  = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    blfb = inner(dzeta,dot_zeta) - inner(grad(dzeta),zeta*Ff*duf)
    return blfb
    
def solve_initial_data(zeta):
    Qvec        = VectorElement("CG", mesh.ufl_cell(), 1) # Function space for solid alpha
    Qscal       = FiniteElement("CG", mesh.ufl_cell(), 1) # Function space for Lagrange multiplier
    Vtemp = FunctionSpace(mesh, MixedElement([Qvec,Qvec,Qscal]))
    Vtemp = FunctionSpace(mesh, MixedElement([Qvec,Qvec]))
    
    qtemp,dqtemp    = Function(Vtemp),TestFunction(Vtemp)
    us,uf    = split(qtemp)
    dus,duf  = split(dqtemp)
    I = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    gtemp = 0
    Etemp = get_energy(us, uf, zeta,gtemp) 
    Etemp += inner(grad(uf),grad(uf))*dx
    Etemp += (zeta - phi*(1/Jf-1))**2*dx
    
    bc0t = DirichletBC(Vtemp.sub(0).sub(0),Constant(0 ),'on_boundary&&(near(x[0],  0,1e-3)||near(x[0],20,1e-3))')      # left & right
    bc1t = DirichletBC(Vtemp.sub(0).sub(1),Constant(0 ),'on_boundary&&(near(x[1],-40,1e-3)||near(x[1],40,1e-3))') # top & bottom
    bc2t = DirichletBC(Vtemp.sub(1).sub(0),Constant(0 ),'on_boundary&&(near(x[0],  0,1e-3)||near(x[0],20,1e-3))')      # left & right
    bc3t = DirichletBC(Vtemp.sub(1).sub(1),Constant(0 ),'on_boundary&&(near(x[1],-40,1e-3)||near(x[1],40,1e-3))') # top & bottom
    
    bct      = [bc0t,bc1t,bc2t,bc3t]

    Restemp =  derivative(Etemp,qtemp,dqtemp)  
    solve(Restemp==0, qtemp, bct,solver_parameters={"newton_solver":{"linear_solver"      : "lu",
                                                    "absolute_tolerance" : 1e-7,
                                                    "maximum_iterations" : 30 }}) 
    return qtemp

# single time step
def evolve(old_q, dt):
    
    Q = old_q.function_space()
    
    q,dq                           = Function(Q),TestFunction(Q)
    us,uf,zeta,muz                 = split(q)
    dus,duf,dzeta,dmuz             = split(dq)
    old_us,old_uf,old_zeta,old_muz = split(old_q)
    
    dot_us =   (us-old_us)
    dot_uf =   (uf-old_uf)
    dot_zeta = (zeta-old_zeta)
    
    E = get_energy(us, uf, zeta,g0) 
    
    Res  =   dt*a(muz,dmuz)*dx
    Res +=   b(old_uf,old_zeta,dot_zeta,dot_uf,dmuz)*dx + dt*b(old_uf,old_zeta,dzeta,duf,muz)*dx
    Res += - s(old_us,old_uf,old_zeta,dot_us,dot_uf,dus,duf)*dx
    Res += - dt*derivative(E,q,dq)

    bc0 = DirichletBC(Q.sub(0).sub(0),Constant(0), 'on_boundary && (near(x[0],0,1e-3)   || near(x[0],20,1e-3))')      # left & right
    bc1 = DirichletBC(Q.sub(0).sub(1),Constant(0), 'on_boundary && (near(x[1],-40,1e-3) || near(x[1],40,1e-3))') # top & bottom
    bc2 = DirichletBC(Q.sub(1).sub(0),Constant(0), 'on_boundary && (near(x[0],0,1e-3)   || near(x[0],20,1e-3))')      # left & right
    bc3 = DirichletBC(Q.sub(1).sub(1),Constant(0), 'on_boundary && (near(x[1],-40,1e-3) || near(x[1],40,1e-3))') # top & bottom
    bc4 = DirichletBC(Q.sub(2)       ,Constant(z0),'on_boundary && (near(x[1],-40,1e-3) || near(x[1],40,1e-3))') # top & bottom
    
    bc      = [bc0,bc1,bc2,bc3,bc4]
    
    q.assign(old_q)
    solve(Res==0, q, bc) # ,solver_parameters={"newton_solver":{"linear_solver"      : "lu",
                         #                                  "absolute_tolerance" : 1e-8,
                         #                                 "maximum_iterations" : 20 }}) 
    ERG = assemble(E)
    return q,ERG        

In [4]:
######################
# MAIN
######################
print("here we go")

n_steps = 10000
pars    = read_dictionary(fout + 'parameters.json')

mu      = pars['mu']
lmbda   = pars['lmbda']
compr   = pars['compr']
mob     = pars['mob']
Mdarcy  = pars['Mdarcy']
viscs   = pars['viscs']
viscf   = pars['viscf']
phi     = pars['phi']
g0      = pars['g0']
tau0    = pars['tau0']
eps     = pars['eps']
eps1    = pars['eps1']
alpha   = pars['alpha']
z0      = pars['z0']
mag     = pars['mag']


printcount = 0
# tau0 = tau0 / 2
# g0   = g0/2
dt_small = 1e-8
# compr *= 10
# lmbda = mu * 3

printcount = 0

if printcount == 0:
    
    it0 = 0
    H   = 20
    
    mesh   = refine_circle(10,30)    
    Q      = get_space(mesh)
    zeta0  = Expression('z0+mag*exp(-alpha*((pow(x[0]-10,2))+(pow(x[1]-30,2))))',z0=z0,alpha=alpha,mag=mag,degree=3)
    zi     = project(zeta0,FunctionSpace(mesh,'CG',1))
    
    print('prepare initial data')
    q_init = solve_initial_data(zi)
    print('done')
    
    old_q = Function(Q)
    assign(old_q.sub(0),q_init.sub(0))
    assign(old_q.sub(1),q_init.sub(1))
    assign(old_q.sub(2),project(zi,FunctionSpace(mesh,'CG',1)))
    
    t        = 0
    
    print('perform small time step')
    q,ERG = evolve(old_q, dt_small)
    old_q.assign(q)
    print('done')
    output_state(mesh,old_q,fout + 'solution' + str(printcount),it0,t)
        
else:
    
    print('continue with data at output step ',printcount)
    mesh,old_q,it0,t0 = read_state(fout + 'solution' + str(printcount))
    t=t0

for it in range(it0,n_steps+1):
    dt = tau0
    t += dt
    
    q,ERG = evolve(old_q, dt)
    old_q.assign(q)

    print("iter: %d t: %4.3f ERG: %8.6f" % (it,t,ERG))

    if (it % 10 == 0):
        printcount += 1
        output_state(mesh,q,fout + 'solution' + str(printcount),it,t)
        
        
    
    if (it % 100 == 99):
        old_q,mesh = refine_mesh(q,mesh)
        q,ERG = evolve(old_q, dt_small)
        old_q.assign(q)
        t += dt_small

here we go
prepare initial data
done
perform small time step
done
iter: 0 t: 20.000 ERG: -9.283801
iter: 1 t: 40.000 ERG: -9.289527
iter: 2 t: 60.000 ERG: -9.295223
iter: 3 t: 80.000 ERG: -9.300896
iter: 4 t: 100.000 ERG: -9.306548
iter: 5 t: 120.000 ERG: -9.312181
iter: 6 t: 140.000 ERG: -9.317795
iter: 7 t: 160.000 ERG: -9.323392
iter: 8 t: 180.000 ERG: -9.328971
iter: 9 t: 200.000 ERG: -9.334533
iter: 10 t: 220.000 ERG: -9.340078
iter: 11 t: 240.000 ERG: -9.345605
iter: 12 t: 260.000 ERG: -9.351116
iter: 13 t: 280.000 ERG: -9.356610
iter: 14 t: 300.000 ERG: -9.362087
iter: 15 t: 320.000 ERG: -9.367548
iter: 16 t: 340.000 ERG: -9.372992
iter: 17 t: 360.000 ERG: -9.378419
iter: 18 t: 380.000 ERG: -9.383830
iter: 19 t: 400.000 ERG: -9.389225
iter: 20 t: 420.000 ERG: -9.394604
iter: 21 t: 440.000 ERG: -9.399966
iter: 22 t: 460.000 ERG: -9.405313
iter: 23 t: 480.000 ERG: -9.410643
iter: 24 t: 500.000 ERG: -9.415958
iter: 25 t: 520.000 ERG: -9.421257
iter: 26 t: 540.000 ERG: -9.426540
ite

iter: 220 t: 4420.000 ERG: -10.309088
iter: 221 t: 4440.000 ERG: -10.313310
iter: 222 t: 4460.000 ERG: -10.317533
iter: 223 t: 4480.000 ERG: -10.321754
iter: 224 t: 4500.000 ERG: -10.325975
iter: 225 t: 4520.000 ERG: -10.330196
iter: 226 t: 4540.000 ERG: -10.334416
iter: 227 t: 4560.000 ERG: -10.338635
iter: 228 t: 4580.000 ERG: -10.342854
iter: 229 t: 4600.000 ERG: -10.347072
iter: 230 t: 4620.000 ERG: -10.351290
iter: 231 t: 4640.000 ERG: -10.355508
iter: 232 t: 4660.000 ERG: -10.359725
iter: 233 t: 4680.000 ERG: -10.363941
iter: 234 t: 4700.000 ERG: -10.368157
iter: 235 t: 4720.000 ERG: -10.372373
iter: 236 t: 4740.000 ERG: -10.376588
iter: 237 t: 4760.000 ERG: -10.380803
iter: 238 t: 4780.000 ERG: -10.385017
iter: 239 t: 4800.000 ERG: -10.389231
iter: 240 t: 4820.000 ERG: -10.393445
iter: 241 t: 4840.000 ERG: -10.397659
iter: 242 t: 4860.000 ERG: -10.401872
iter: 243 t: 4880.000 ERG: -10.406084
iter: 244 t: 4900.000 ERG: -10.410297
iter: 245 t: 4920.000 ERG: -10.414509
iter: 246 t:

iter: 435 t: 8720.000 ERG: -11.216493
iter: 436 t: 8740.000 ERG: -11.220764
iter: 437 t: 8760.000 ERG: -11.225036
iter: 438 t: 8780.000 ERG: -11.229309
iter: 439 t: 8800.000 ERG: -11.233583
iter: 440 t: 8820.000 ERG: -11.237857
iter: 441 t: 8840.000 ERG: -11.242131
iter: 442 t: 8860.000 ERG: -11.246407
iter: 443 t: 8880.000 ERG: -11.250683
iter: 444 t: 8900.000 ERG: -11.254960
iter: 445 t: 8920.000 ERG: -11.259238
iter: 446 t: 8940.000 ERG: -11.263516
iter: 447 t: 8960.000 ERG: -11.267795
iter: 448 t: 8980.000 ERG: -11.272075
iter: 449 t: 9000.000 ERG: -11.276355
iter: 450 t: 9020.000 ERG: -11.280636
iter: 451 t: 9040.000 ERG: -11.284918
iter: 452 t: 9060.000 ERG: -11.289201
iter: 453 t: 9080.000 ERG: -11.293484
iter: 454 t: 9100.000 ERG: -11.297768
iter: 455 t: 9120.000 ERG: -11.302053
iter: 456 t: 9140.000 ERG: -11.306338
iter: 457 t: 9160.000 ERG: -11.310624
iter: 458 t: 9180.000 ERG: -11.314911
iter: 459 t: 9200.000 ERG: -11.319199
iter: 460 t: 9220.000 ERG: -11.323488
iter: 461 t:

iter: 645 t: 12920.000 ERG: -12.126555
iter: 646 t: 12940.000 ERG: -12.131014
iter: 647 t: 12960.000 ERG: -12.135474
iter: 648 t: 12980.000 ERG: -12.139935
iter: 649 t: 13000.000 ERG: -12.144397
iter: 650 t: 13020.000 ERG: -12.148860
iter: 651 t: 13040.000 ERG: -12.153324
iter: 652 t: 13060.000 ERG: -12.157789
iter: 653 t: 13080.000 ERG: -12.162255
iter: 654 t: 13100.000 ERG: -12.166722
iter: 655 t: 13120.000 ERG: -12.171189
iter: 656 t: 13140.000 ERG: -12.175658
iter: 657 t: 13160.000 ERG: -12.180128
iter: 658 t: 13180.000 ERG: -12.184598
iter: 659 t: 13200.000 ERG: -12.189069
iter: 660 t: 13220.000 ERG: -12.193542
iter: 661 t: 13240.000 ERG: -12.198015
iter: 662 t: 13260.000 ERG: -12.202489
iter: 663 t: 13280.000 ERG: -12.206964
iter: 664 t: 13300.000 ERG: -12.211440
iter: 665 t: 13320.000 ERG: -12.215917
iter: 666 t: 13340.000 ERG: -12.220394
iter: 667 t: 13360.000 ERG: -12.224873
iter: 668 t: 13380.000 ERG: -12.229352
iter: 669 t: 13400.000 ERG: -12.233832
iter: 670 t: 13420.000 ER

iter: 854 t: 17100.000 ERG: -13.080699
iter: 855 t: 17120.000 ERG: -13.085386
iter: 856 t: 17140.000 ERG: -13.090074
iter: 857 t: 17160.000 ERG: -13.094764
iter: 858 t: 17180.000 ERG: -13.099455
iter: 859 t: 17200.000 ERG: -13.104148
iter: 860 t: 17220.000 ERG: -13.108843
iter: 861 t: 17240.000 ERG: -13.113539
iter: 862 t: 17260.000 ERG: -13.118236
iter: 863 t: 17280.000 ERG: -13.122935
iter: 864 t: 17300.000 ERG: -13.127636
iter: 865 t: 17320.000 ERG: -13.132338
iter: 866 t: 17340.000 ERG: -13.137041
iter: 867 t: 17360.000 ERG: -13.141746
iter: 868 t: 17380.000 ERG: -13.146453
iter: 869 t: 17400.000 ERG: -13.151161
iter: 870 t: 17420.000 ERG: -13.155871
iter: 871 t: 17440.000 ERG: -13.160583
iter: 872 t: 17460.000 ERG: -13.165296
iter: 873 t: 17480.000 ERG: -13.170011
iter: 874 t: 17500.000 ERG: -13.174727
iter: 875 t: 17520.000 ERG: -13.179445
iter: 876 t: 17540.000 ERG: -13.184165
iter: 877 t: 17560.000 ERG: -13.188886
iter: 878 t: 17580.000 ERG: -13.193609
iter: 879 t: 17600.000 ER

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve nonlinear system with NewtonSolver.
*** Reason:  Newton solver did not converge because maximum number of iterations reached.
*** Where:   This error was encountered inside NewtonSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------


In [ ]:
plt.figure(figsize=(5*3,20*3))

us,uf,zeta,muz                 = split(q)

# c = plot(old_q.sub(2))
c = plot(zeta)
# c = plot(sqrt(inner(us,us)))
#plt.colorbar(c)

plot(mesh,linewidth=0.5)
plt.colorbar(c)
# q1,m1 = refine_mesh(old_q,mesh)
# 
# plt.figure(figsize=(10,10))
# plot(q1.sub(2))
# plot(m1,linewidth=0.2)

